# S3W9D4: 法律文档 ETL 管道 (The Pipeline)

**今日目标**：
编写 `src/rag/ingest.py`，实现 **Extract (读取) -> Transform (切分+清洗) -> Load (向量化+存储)** 的全自动化流程。

**1准备数据文件**
请在你的项目根目录下创建 `data` 文件夹，并放入一个测试用的法律文档。

```bash
mkdir -p data
# 创建一个模拟的刑法文本文件
touch data/penal_code.txt

```

请将以下内容复制到 `data/penal_code.txt` 中（这是真实刑法的片段）：

```text
第二百六十四条 【盗窃罪】
盗窃公私财物，数额较大的，或者多次盗窃、入户盗窃、携带凶器盗窃、扒窃的，处三年以下有期徒刑、拘役或者管制，并处或者单处罚金；数额巨大或者有其他严重情节的，处三年以上十年以下有期徒刑，并处罚金；数额特别巨大或者有其他特别严重情节的，处十年以上有期徒刑或者无期徒刑，并处罚金或者没收财产。

第二百六十六条 【诈骗罪】
诈骗公私财物，数额较大的，处三年以下有期徒刑、拘役或者管制，并处或者单处罚金；数额巨大或者有其他严重情节的，处三年以上十年以下有期徒刑，并处罚金；数额特别巨大或者有其他特别严重情节的，处十年以上有期徒刑或者无期徒刑，并处罚金或者没收财产。

第二百三十四条 【故意伤害罪】
故意伤害他人身体的，处三年以下有期徒刑、拘役或者管制。
犯前款罪，致人重伤的，处三年以上十年以下有期徒刑；致人死亡或者以特别残忍手段致人重伤造成严重残疾的，处十年以上有期徒刑、无期徒刑或者死刑。本法另有规定的，依照规定。
```

## 1 理论知识讲解 (CS Principle)

**ETL** 是数据工程的基石：

1. **Extract (抽取)**：
    * 从文件系统读取 `.txt` / `.pdf`。
    * *挑战*：编码格式（UTF-8 vs GBK）、文件读取异常。


2. **Transform (转换)**：
    * **Cleaning**：去除多余的空格、换行符。
    * **Chunking**：调用 Day 1 的切分器。
    * **Metadata Injection**：**这步最关键！** 原始文本切分后会丢失上下文。我们需要手动把文件名、章节名作为 `metadata` 附在每一个 Chunk 上（例如 `{"source": "penal_code.txt"}`），否则将来检索出来你都不知道这段话出自哪里。


3. **Load (加载)**：
    * 调用 Day 3 的 `VectorDB.add_documents()`。

## 2. 代码实现 (The Ingestion Script)

我们将利用之前封装好的模块，像搭积木一样构建管道。

**创建/编辑文件**：`src/rag/ingest.py`

```python
import os
import sys
# 路径处理：确保能导入 src 下的模块
sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__), "../../")))

from src.rag.etl import TextChunker
from src.rag.vector_db import VectorDB

def load_and_process_file(file_path: str):
    """
    Step 1: Extract - 读取文件
    """
    print(f"📄 Processing file: {file_path}")
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read()
    except Exception as e:
        print(f"Error reading file: {e}")
        return [], []
    
    # 简单的元数据提取 (实际工程中可能需要解析文件名或正则提取章节号)
    file_name = os.path.basename(file_path)
    
    """
    Step 2: Transform - 切分与清洗
    """
    # 初始化我们 Day 1 写的切分器
    chunker = TextChunker(chunk_size=300, chunk_overlap=50)
    chunks = chunker.split(text)
    
    # 构造 Metadata
    metadatas = []
    for i, chunk in enumerate(chunks):
        metadatas.append({
            "source": file_name,
            "chunk_id": i,
            "length": len(chunk)
        })
    
    print(f"✂️  Split into {len(chunks)} chunks.")
    return chunks, metadatas

def main():
    # 配置
    DATA_DIR = "data"
    DB_PATH = "chroma_db_data" # 注意这里要和 Day 3 保持一致
    
    # 初始化组件
    print("🚀 Starting ETL Pipeline...")
    
    # 初始化 DB (Day 3 的组件)
    # 注意：如果目录存在，它会加载旧数据；如果不存在，会新建
    vector_db = VectorDB(persist_path=DB_PATH)
    
    # 遍历 data 目录下的所有 txt 文件
    for filename in os.listdir(DATA_DIR):
        if filename.endswith(".txt"):
            file_path = os.path.join(DATA_DIR, filename)
            
            # 1. 处理数据
            chunks, metadatas = load_and_process_file(file_path)
            
            if chunks:
                # 2. Step 3: Load - 存入数据库
                print(f"💾 Ingesting to VectorDB...")
                vector_db.add_documents(chunks, metadatas)
                print(f"✅ Successfully ingested {filename}")

    print("🎉 ETL Pipeline Completed!")

    # --- 验证环节 ---
    print("\n🔍 Verifying with a test query...")
    results = vector_db.search("故意伤害致死怎么判？", top_k=1)
    for res in results:
        print(f"Answer found in [{res['metadata']['source']}]:")
        print(f"Content: {res['text'][:50]}...") # 只打印前50个字
        print(f"Distance: {res['distance']:.4f}")

if __name__ == "__main__":
    main()

```

---

## 3. 运行与验证

在终端中运行：

```bash
python src/rag/ingest.py

```

**预期输出**：

1. 应该能看到 Loading Model (from local cache) 的日志。
2. 看到 `Processing file: .../data/penal_code.txt`。
3. 看到 `Split into ... chunks`。
4. 最后验证环节，针对问题“故意伤害致死”，应该能检索出包含“第二百三十四条”的那段文本。

---

## 4. 今日 LeetCode 练习

我们今天处理的是 **Pipeline (管道)**，数据像流水一样经过一个个关卡。这和 **Queue (队列)** 的先进先出特性非常像。

你之前的列表中有这道题，非常适合今天练习。

* **题目**：[225. Implement Stack using Queues (用队列实现栈)](https://leetcode.cn/problems/implement-stack-using-queues/)
* **状态**：在你的列表中 (题号 225)。
* **关联**：Day 2 我们做了“最小栈”，Day 3 做了链表归并。今天挑战一下用 FIFO (队列) 的积木去搭 LIFO (栈) 的城堡。
* **核心难点**：
* 队列是“两头通”（一头进一头出）。
* 栈是“死胡同”（只能从一头进出）。
* **思路**：当你 `push` 一个新元素进队列时，它排在队尾。为了让它变成“栈顶”（队头），你需要把前面的所有老元素取出来，重新排到它后面去。



**请尝试实现并提交。**

---

## 5. 今日任务总结

* [ ] 创建了 `data/penal_code.txt` 样本数据。
* [ ] 编写了 `src/rag/ingest.py`，打通了 `Chunker` -> `Embedding` -> `VectorDB` 的全流程。
* [ ] 验证了数据入库后能被检索到。
* [ ] 练习 LeetCode 225。

**这一步非常关键**。现在你的 AI 已经不再是空架子了，它肚子里真的有了刑法知识。
明天（Day 5），我们将编写一个 **Web 界面 (Streamlit) 或者 CLI 工具**，把 LLM (大脑) 接进来，让它利用这些知识真正回答用户的问题（RAG 的最终形态）。

准备好开始写代码了吗？